# Risk Platform Business User Analysis

**Purpose**: This notebook demonstrates how business users can leverage the Risk Platform's capabilities for comprehensive portfolio risk analysis.

**Prerequisites**: 
- Access to JupyterHub environment
- Risk Platform API access
- Valid portfolio data

**Author**: Risk Platform Team  
**Last Updated**: September 2025

## 📊 Environment Setup

In [ ]:
# Import required libraries
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully")

In [ ]:
# Configuration - Update these endpoints for your environment
RISK_API_BASE_URL = "http://fastapi-service.default.svc.cluster.local/api/v1"
# Alternative for external access: "https://your-alb-endpoint/api/v1"

# API Client Configuration
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
    # Add authentication headers when implemented
    # 'Authorization': 'Bearer YOUR_TOKEN_HERE'
}

print(f"🔧 API Base URL: {RISK_API_BASE_URL}")
print(f"📡 Headers configured: {list(headers.keys())}")

## 🏥 Health Check & API Validation

In [ ]:
# Check API health
try:
    health_response = requests.get(f"{RISK_API_BASE_URL.replace('/api/v1', '')}/health", headers=headers)
    if health_response.status_code == 200:
        print("✅ Risk API is healthy and accessible")
        print(f"📊 Response: {health_response.json()}")
    else:
        print(f"⚠️ API health check failed: {health_response.status_code}")
except Exception as e:
    print(f"❌ Cannot connect to Risk API: {str(e)}")
    print("💡 Make sure you're running this from within the JupyterHub environment")

## 📈 Portfolio Data Retrieval

In [ ]:
# Get available portfolios
try:
    portfolios_response = requests.get(f"{RISK_API_BASE_URL}/portfolios", headers=headers)
    if portfolios_response.status_code == 200:
        portfolios = portfolios_response.json()
        print(f"✅ Found {len(portfolios)} portfolios")
        
        # Display portfolio summary
        portfolio_df = pd.DataFrame(portfolios)
        print("\n📊 Portfolio Overview:")
        display(portfolio_df.head())
    else:
        print(f"⚠️ Could not retrieve portfolios: {portfolios_response.status_code}")
        # Create sample data for demonstration
        portfolios = [
            {"id": "portfolio_1", "name": "Tech Growth Portfolio", "total_value": 1000000},
            {"id": "portfolio_2", "name": "Conservative Bonds", "total_value": 500000},
            {"id": "portfolio_3", "name": "Mixed Equity Fund", "total_value": 750000}
        ]
        portfolio_df = pd.DataFrame(portfolios)
        print("\n📊 Using sample portfolio data:")
        display(portfolio_df)
        
except Exception as e:
    print(f"❌ Error retrieving portfolios: {str(e)}")
    portfolios = []

## 🎯 Risk Metrics Analysis

In [ ]:
# Define sample portfolio for risk calculation
sample_portfolio = {
    "portfolio_id": "portfolio_1",
    "positions": [
        {"symbol": "AAPL", "quantity": 100, "price": 150.0},
        {"symbol": "GOOGL", "quantity": 50, "price": 2800.0},
        {"symbol": "MSFT", "quantity": 75, "price": 300.0},
        {"symbol": "TSLA", "quantity": 25, "price": 800.0},
        {"symbol": "NVDA", "quantity": 30, "price": 450.0}
    ],
    "time_horizon": "1d",
    "confidence_level": 0.95
}

print("🎯 Sample Portfolio Configuration:")
positions_df = pd.DataFrame(sample_portfolio['positions'])
positions_df['market_value'] = positions_df['quantity'] * positions_df['price']
positions_df['weight'] = positions_df['market_value'] / positions_df['market_value'].sum()

display(positions_df)
print(f"\n💰 Total Portfolio Value: ${positions_df['market_value'].sum():,.2f}")

In [ ]:
# Calculate portfolio risk metrics
try:
    risk_response = requests.post(
        f"{RISK_API_BASE_URL}/risk/calculate", 
        headers=headers,
        json=sample_portfolio
    )
    
    if risk_response.status_code == 200:
        risk_metrics = risk_response.json()
        print("✅ Risk metrics calculated successfully")
        
        # Display key risk metrics
        print("\n📊 Key Risk Metrics:")
        print(f"🎯 Value at Risk (VaR): ${risk_metrics.get('var', 'N/A')}")
        print(f"📈 Expected Shortfall: ${risk_metrics.get('expected_shortfall', 'N/A')}")
        print(f"📊 Portfolio Volatility: {risk_metrics.get('volatility', 'N/A'):.2%}")
        print(f"🔄 Beta: {risk_metrics.get('beta', 'N/A')}")
        print(f"📉 Maximum Drawdown: {risk_metrics.get('max_drawdown', 'N/A'):.2%}")
        
    else:
        print(f"⚠️ Risk calculation failed: {risk_response.status_code}")
        # Create sample risk metrics for demonstration
        risk_metrics = {
            "var": 45000,
            "expected_shortfall": 67500,
            "volatility": 0.18,
            "beta": 1.15,
            "max_drawdown": 0.12,
            "sharpe_ratio": 1.25
        }
        print("\n📊 Using sample risk metrics:")
        print(f"🎯 Value at Risk (VaR): ${risk_metrics['var']:,.2f}")
        print(f"📈 Expected Shortfall: ${risk_metrics['expected_shortfall']:,.2f}")
        print(f"📊 Portfolio Volatility: {risk_metrics['volatility']:.2%}")
        print(f"🔄 Beta: {risk_metrics['beta']:.2f}")
        print(f"📉 Maximum Drawdown: {risk_metrics['max_drawdown']:.2%}")
        print(f"⭐ Sharpe Ratio: {risk_metrics['sharpe_ratio']:.2f}")
        
except Exception as e:
    print(f"❌ Error calculating risk metrics: {str(e)}")
    risk_metrics = {}

## 📊 Risk Visualization Dashboard

In [ ]:
# Create comprehensive risk dashboard
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Portfolio Risk Analysis Dashboard', fontsize=16, fontweight='bold')

# 1. Portfolio Composition
ax1.pie(positions_df['market_value'], labels=positions_df['symbol'], autopct='%1.1f%%', startangle=90)
ax1.set_title('Portfolio Composition by Market Value', fontweight='bold')

# 2. Risk Metrics Bar Chart
if risk_metrics:
    metrics = ['VaR', 'Expected Shortfall', 'Beta', 'Sharpe Ratio']
    values = [risk_metrics.get('var', 0)/1000, risk_metrics.get('expected_shortfall', 0)/1000, 
              risk_metrics.get('beta', 0), risk_metrics.get('sharpe_ratio', 0)]
    
    bars = ax2.bar(metrics, values, color=['red', 'orange', 'blue', 'green'])
    ax2.set_title('Key Risk Metrics', fontweight='bold')
    ax2.set_ylabel('Values')
    
    # Add value labels on bars
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{value:.2f}', ha='center', va='bottom')

# 3. Volatility Analysis (simulated historical data)
dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
np.random.seed(42)
returns = np.random.normal(0.0008, 0.02, len(dates))  # Daily returns
cumulative_returns = (1 + pd.Series(returns, index=dates)).cumprod()

ax3.plot(dates, cumulative_returns, linewidth=2, color='navy')
ax3.set_title('Simulated Portfolio Performance (2024)', fontweight='bold')
ax3.set_ylabel('Cumulative Returns')
ax3.grid(True, alpha=0.3)

# 4. Risk Distribution
risk_scenarios = np.random.normal(0, risk_metrics.get('volatility', 0.18), 1000)
ax4.hist(risk_scenarios, bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
ax4.axvline(x=np.percentile(risk_scenarios, 5), color='red', linestyle='--', 
           label=f'5% VaR: {np.percentile(risk_scenarios, 5):.3f}')
ax4.set_title('Risk Distribution (Monte Carlo)', fontweight='bold')
ax4.set_xlabel('Returns')
ax4.set_ylabel('Frequency')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Risk dashboard generated successfully")

## 📈 Market Data Analysis

In [ ]:
# Retrieve market data for portfolio positions
symbols = positions_df['symbol'].tolist()
market_data_request = {
    "symbols": symbols,
    "start_date": "2024-01-01",
    "end_date": "2024-12-31",
    "data_type": "daily"
}

try:
    market_response = requests.post(
        f"{RISK_API_BASE_URL}/market/data", 
        headers=headers,
        json=market_data_request
    )
    
    if market_response.status_code == 200:
        market_data = market_response.json()
        print("✅ Market data retrieved successfully")
        print(f"📊 Data points: {len(market_data.get('data', []))}")
    else:
        print(f"⚠️ Market data request failed: {market_response.status_code}")
        # Generate sample market data for demonstration
        print("📊 Generating sample market data for analysis...")
        
except Exception as e:
    print(f"❌ Error retrieving market data: {str(e)}")
    print("📊 Using simulated market data for demonstration")

# Create sample correlation matrix
np.random.seed(42)
correlation_matrix = np.random.uniform(0.3, 0.9, (len(symbols), len(symbols)))
# Make it symmetric
correlation_matrix = (correlation_matrix + correlation_matrix.T) / 2
np.fill_diagonal(correlation_matrix, 1.0)

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(pd.DataFrame(correlation_matrix, index=symbols, columns=symbols), 
            annot=True, cmap='coolwarm', center=0, square=True,
            linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('Portfolio Asset Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("📈 Market analysis completed")

## 📑 Risk Report Generation

In [ ]:
# Generate comprehensive risk report
report_request = {
    "portfolio_id": sample_portfolio["portfolio_id"],
    "report_type": "comprehensive_risk",
    "format": "json",
    "include_charts": True,
    "date_range": {
        "start": "2024-01-01",
        "end": "2024-12-31"
    }
}

try:
    report_response = requests.post(
        f"{RISK_API_BASE_URL}/reports/generate", 
        headers=headers,
        json=report_request
    )
    
    if report_response.status_code == 200:
        report_data = report_response.json()
        print("✅ Risk report generated successfully")
        print(f"📑 Report ID: {report_data.get('report_id', 'N/A')}")
        print(f"📊 Report Status: {report_data.get('status', 'N/A')}")
    else:
        print(f"⚠️ Report generation failed: {report_response.status_code}")
        
except Exception as e:
    print(f"❌ Error generating report: {str(e)}")

# Create summary report
print("\n" + "="*60)
print("📋 PORTFOLIO RISK ANALYSIS SUMMARY")
print("="*60)
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💼 Portfolio ID: {sample_portfolio['portfolio_id']}")
print(f"💰 Total Value: ${positions_df['market_value'].sum():,.2f}")
print(f"🎯 Number of Positions: {len(positions_df)}")
print("\n📊 KEY RISK METRICS:")
if risk_metrics:
    print(f"   • Value at Risk (95%): ${risk_metrics.get('var', 0):,.2f}")
    print(f"   • Expected Shortfall: ${risk_metrics.get('expected_shortfall', 0):,.2f}")
    print(f"   • Portfolio Volatility: {risk_metrics.get('volatility', 0):.2%}")
    print(f"   • Beta: {risk_metrics.get('beta', 0):.2f}")
    print(f"   • Sharpe Ratio: {risk_metrics.get('sharpe_ratio', 0):.2f}")

print("\n🎯 TOP POSITIONS:")
top_positions = positions_df.nlargest(3, 'market_value')
for _, pos in top_positions.iterrows():
    print(f"   • {pos['symbol']}: ${pos['market_value']:,.2f} ({pos['weight']:.1%})")

print("\n⚠️ RISK RECOMMENDATIONS:")
if risk_metrics.get('volatility', 0) > 0.20:
    print("   • Consider reducing portfolio volatility through diversification")
if risk_metrics.get('beta', 1) > 1.2:
    print("   • Portfolio has high market sensitivity - consider defensive positions")
if positions_df['weight'].max() > 0.30:
    print("   • Consider reducing concentration risk in top holdings")

print("\n✅ Analysis completed successfully")
print("="*60)

## 🔄 Next Steps & Business Actions

### 📈 Regular Monitoring
- **Daily**: Monitor VaR and portfolio performance
- **Weekly**: Review position weights and rebalancing needs
- **Monthly**: Comprehensive risk assessment and stress testing

### 🎯 Risk Management Actions
1. **High Risk Alert**: If VaR exceeds 5% of portfolio value
2. **Concentration Review**: If any position exceeds 25% weight
3. **Correlation Check**: Monitor asset correlation changes

### 🔧 Platform Integration
- **Airflow Workflows**: Schedule automated risk calculations
- **Dash Dashboards**: Real-time risk monitoring
- **API Integration**: Connect with external risk systems

### 📚 Additional Resources
- Risk Platform Documentation: `/docs/`
- API Reference: `http://fastapi-service/docs`
- Support: Contact risk platform team